In [1]:
import gym
import ptan
import random
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn

from ignite.engine import Engine

from lib import dqn_model, common

NAME = "03_double"
STATES_TO_EVALUATE = 1000
EVAL_EVERY_FRAME = 100

In [15]:
def calc_loss_double_dqn(batch, net, tgt_net, gamma,
                         device="cpu", double=True):
    states, actions, rewards, dones, next_states = \
        common.unpack_batch(batch)
    
    states_v = torch.tensor(states).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.BoolTensor(dones).to(device)
    
    actions_v = actions_v.unsqueeze(-1)
    state_action_vals = net(states_v).gather(1, actions_v.type(torch.int64))
    state_action_vals = state_action_vals.squeeze(-1)
    with torch.no_grad():
        next_states_v = torch.tensor(next_states).to(device)
        if double:
            next_state_acts = net(next_states_v).max(1)[1]
            next_state_acts = next_state_acts.unsqueeze(-1)
            next_state_vals = tgt_net(next_states_v).gather(
                1, next_state_acts.type(torch.int64)).squeeze(-1)
        else:
            next_state_vals = tgt_net(next_states_v).max(1)[0]
        next_state_vals[done_mask] = 0.0
        exp_sa_vals = next_state_vals.detach() * gamma + rewards_v
    return nn.MSELoss()(state_action_vals, exp_sa_vals)

In [3]:
random.seed(common.SEED)
torch.manual_seed(common.SEED)
params = common.HYPERPARAMS['pong']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make("PongNoFrameskip-v4")
env = ptan.common.wrappers.wrap_dqn(env)
env.seed(common.SEED)

[123, 151010689]

In [4]:
net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)

tgt_net = ptan.agent.TargetNet(net)
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=params.epsilon_start)
epsilon_tracker = common.EpsilonTracker(selector, params)
agent = ptan.agent.DQNAgent(net, selector, device = device)

exp_source = ptan.experience.ExperienceSourceFirstLast(
    env, agent, gamma=params.gamma)
buffer = ptan.experience.ExperienceReplayBuffer(
    exp_source, buffer_size=params.replay_size)
optimizer = optim.Adam(net.parameters(), lr=params.learning_rate)

In [17]:
def process_batch(engine, batch):
    optimizer.zero_grad()
    loss_v = calc_loss_double_dqn(batch, net, tgt_net.target_model,
                                  gamma=params.gamma, device=device,
                                  double=True)
    loss_v.backward()
    optimizer.step()
    epsilon_tracker.frame(engine.state.iteration)
    if engine.state.iteration % params.target_net_sync == 0:
        tgt_net.sync()
    if engine.state.iteration % EVAL_EVERY_FRAME == 0:
        eval_states = getattr(engine.state, "eval_states", None)
        if eval_states is None:
            eval_states = buffer.sample(STATES_TO_EVALUATE)
            eval_states = [np.array(transition.state, copy=False) for transition in eval_states]
            eval_states = np.array(eval_states, copy=False)
            engine.state.eval_states = eval_states
        engine.state.metrics["values"] = \
            common.calc_values_of_states(eval_states, net, device)
    return {
        "loss" : loss_v.item(),
        "epsilon": selector.epsilon,
    }

engine = Engine(process_batch)
common.setup_ignite(engine, params, exp_source, f"{NAME}={True}", extra_metrics=('values',))
engine.run(common.batch_generator(buffer, params.replay_initial, params.batch_size))

Episode 1: reward=-21, steps=818, speed=0.0 f/s, elapsed=0:00:59
Episode 2: reward=-19, steps=1022, speed=0.0 f/s, elapsed=0:00:59
Episode 3: reward=-21, steps=850, speed=0.0 f/s, elapsed=0:00:59
Episode 4: reward=-21, steps=840, speed=0.0 f/s, elapsed=0:00:59
Episode 5: reward=-20, steps=895, speed=0.0 f/s, elapsed=0:00:59
Episode 6: reward=-19, steps=958, speed=0.0 f/s, elapsed=0:00:59
Episode 7: reward=-20, steps=992, speed=0.0 f/s, elapsed=0:00:59
Episode 8: reward=-21, steps=819, speed=0.0 f/s, elapsed=0:00:59
Episode 9: reward=-21, steps=783, speed=0.0 f/s, elapsed=0:00:59
Episode 10: reward=-21, steps=937, speed=0.0 f/s, elapsed=0:00:59
Episode 11: reward=-19, steps=1038, speed=0.0 f/s, elapsed=0:00:59
Episode 12: reward=-20, steps=972, speed=0.0 f/s, elapsed=0:00:59
Episode 13: reward=-21, steps=895, speed=0.0 f/s, elapsed=0:00:59
Episode 14: reward=-19, steps=924, speed=0.0 f/s, elapsed=0:00:59
Episode 15: reward=-19, steps=1034, speed=0.0 f/s, elapsed=0:00:59
Episode 16: rewa

Engine run is terminating due to exception: .


KeyboardInterrupt: 